# Mock population - two box model

In this notebook we generate a mock catalog of GW events

In [1]:
#IMPORT
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import cumtrapz, trapz
import scipy.stats as stats
import time


#PYTHON MODULES
from constants import *
import gwcosmo
import gwpop
import gwutils
import utils
import sensitivity_curves as sc

#PLOTS
import sys
dir_base=sys.path[0]
dir_out=dir_base+"/plots_spectral_sirens/"

#from matplotlib.ticker import ScalarFormatter
#%config InlineBackend.figure_format = 'retina'
#plt.rc('text', usetex=True)
#plt.rc('font', family='serif')
fontSz = 15
fontsz = 13
fontssz = 11

new_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf']

---
We load our fiducial universe parameters

In [2]:
from fiducial_universe import *

edge_1_fid = 5.0
edge_2_fid = 76.25
width_1_fid = 23.75
width_2_fid = width_1_fid
filter_fid = 1.0

In [3]:
fmin = 10.
Tobs = 1.
detector = 'O5'
det_Sc = sc.O5
based = 'ground'
snr_th = 8.0

We define the source populations in source masses and redshift.

In this case we choose:
- Refshift: SFR like function
- Primary mass: smooth two-box model
- Secondary mass: uniform in mass ratio

In [4]:
def mock_source_parameters(n_sources,H0,Om0,edge_1,width_1,edge_2,width_2,filter,zp,alpha_z,beta):
    zs = np.linspace(0.01,10,1000)
    cdf_z = cumtrapz(gwcosmo.diff_comoving_volume_approx(zs,H0,Om0)*gwpop.rate_z(zs,zp,alpha_z,beta)/(1+zs),zs,initial=0)
    cdf_z /= cdf_z[-1]
    masses = np.linspace(tmp_min,tmp_max,1000)
    cdf_m1 = cumtrapz(gwpop.two_box(masses,edge_1,width_1,edge_2,width_2,filter),masses,initial=0)
    cdf_m1 /= cdf_m1[-1]

    z_mock = utils.inverse_transf_sampling(cdf_z,zs,n_sources)
    m1_mock = utils.inverse_transf_sampling(cdf_m1,masses,n_sources)
    q_mock = np.random.uniform(0,1,n_sources)
    m2_mock = m1_mock * q_mock
    dL_mock = gwcosmo.dL_approx(z_mock,H0,Om0)
    m1z_mock = (1 + z_mock) * m1_mock
    m2z_mock = (1 + z_mock) * m2_mock
    return m1z_mock, m2z_mock, dL_mock

When computing the mock detections we follow the recipe of https://arxiv.org/pdf/1911.05882.pdf to estimate the observed posteriors

In [5]:
def mock_population(n_sources,n_detections,n_samples,H0,Om0,edge_1,width_1,edge_2,width_2,filter,zp,alpha_z,beta,snr_th,fmin,Tobs,detector,*args):
    #n_sources : number of sources to run code
    #n_samples : number of posterior samples per detection
    
    #Mock source paramters
    ww = np.linspace(0.0,1.0,1000)
    cdf_ww = 1.0-gwutils.pw_hl(ww)
    m1z_mock_pop, m2z_mock_pop, dL_mock_pop = mock_source_parameters(n_sources,H0,Om0,edge_1,width_1,edge_2,width_2,filter,zp,alpha_z,beta)
    
    #SNR calcultion: optimal SNR -> true SNR -> observed SNR
    snr_opt_mock_pop = gwutils.vsnr(m1z_mock_pop,m2z_mock_pop,dL_mock_pop,fmin,Tobs,det_Sc,based)
    w_mock_pop = utils.inverse_transf_sampling(cdf_ww,ww,n_sources) #random draw
    snr_true_mock_pop = snr_opt_mock_pop*w_mock_pop
    snr_obs_mock_pop = gwutils.observed_snr(snr_true_mock_pop)

    #Detected population
    detected = snr_obs_mock_pop > snr_th
    snr_opt_mock = snr_opt_mock_pop[detected]
    snr_true_mock = snr_true_mock_pop[detected]
    snr_obs_mock = snr_obs_mock_pop[detected]
    m1z_mock = m1z_mock_pop[detected]
    m2z_mock = m2z_mock_pop[detected]
    dL_mock = dL_mock_pop[detected]
    
    snr_opt_mock = snr_opt_mock_pop[detected]
    snr_true_mock = snr_true_mock_pop[detected]
    snr_obs_mock = snr_obs_mock_pop[detected]
    m1z_mock = m1z_mock_pop[detected]
    m2z_mock = m2z_mock_pop[detected]
    dL_mock = dL_mock_pop[detected]
    
    while np.size(snr_opt_mock) < n_detections:    
        m1z_mock_pop, m2z_mock_pop, dL_mock_pop = mock_source_parameters(n_sources,H0,Om0,edge_1,width_1,edge_2,width_2,filter,zp,alpha_z,beta)
        snr_opt_mock_pop = gwutils.vsnr(m1z_mock_pop,m2z_mock_pop,dL_mock_pop,fmin,Tobs,det_Sc,based)
        w_mock_pop = utils.inverse_transf_sampling(cdf_ww,ww,n_sources) #random draw
        snr_true_mock_pop = snr_opt_mock_pop*w_mock_pop
        snr_obs_mock_pop = gwutils.observed_snr(snr_true_mock_pop)
    
        detected = snr_obs_mock_pop > snr_th
        snr_opt_mock_add = snr_opt_mock_pop[detected]
        snr_true_mock_add = snr_true_mock_pop[detected]
        snr_obs_mock_add = snr_obs_mock_pop[detected]
        m1z_mock_add = m1z_mock_pop[detected]
        m2z_mock_add = m2z_mock_pop[detected]
        dL_mock_add = dL_mock_pop[detected]
    
        snr_opt_mock = np.append(snr_opt_mock,snr_opt_mock_add)
        snr_true_mock = np.append(snr_true_mock,snr_true_mock_add)
        snr_obs_mock = np.append(snr_obs_mock,snr_obs_mock_add)
        m1z_mock = np.append(m1z_mock,m1z_mock_add)
        m2z_mock = np.append(m2z_mock,m2z_mock_add)
        dL_mock = np.append(dL_mock,dL_mock_add)
    
    snr_opt_mock_det = snr_opt_mock[0:n_detections]
    snr_true_mock_det = snr_true_mock[0:n_detections]
    snr_obs_mock_det = snr_obs_mock[0:n_detections]
    m1z_mock_det = m1z_mock[0:n_detections] 
    m2z_mock_det = m2z_mock[0:n_detections]
    dL_mock_det = dL_mock[0:n_detections]
    
    m1z_mock_samples = np.zeros((n_detections,n_samples))
    m2z_mock_samples = np.zeros((n_detections,n_samples))
    dL_mock_samples = np.zeros((n_detections,n_samples))
    pdraw_mock_samples = np.zeros((n_detections,n_samples))
    for i in range(n_detections):
        m1z_mock_samples[i,:], m2z_mock_samples[i,:], dL_mock_samples[i,:], pdraw_mock_samples[i,:] = gwutils.observed_posteriors(n_samples,
                                                          m1z_mock_det[i],
                                                          m2z_mock_det[i],
                                                          dL_mock_det[i],
                                                          snr_opt_mock_det[i],
                                                          snr_true_mock_det[i],
                                                          snr_obs_mock_det[i],
                                                          snr_th,
                                                          fmin,
                                                          Tobs,
                                                          detector,
                                                          *args)
        
    return m1z_mock_samples, m2z_mock_samples, dL_mock_samples, pdraw_mock_samples

In [8]:
n_detections = 1000
n_samples = 20
n_sources = n_detections*10

m1z_mock_samples = np.zeros((n_detections,n_samples))
m2z_mock_samples = np.zeros((n_detections,n_samples))
dL_mock_samples = np.zeros((n_detections,n_samples))
pdraw_mock_samples = np.zeros((n_detections,n_samples))

starttime = time.time()

m1z_mock_samples,m2z_mock_samples,dL_mock_samples,pdraw_mock_samples = mock_population(n_sources,
                                        n_detections,
                                        n_samples,
                                        H0_fid,
                                        Om0_fid,
                                        edge_1_fid,
                                        width_1_fid,
                                        edge_2_fid,
                                        width_2_fid,
                                        filter_fid,
                                        zp_fid,
                                        alpha_z_fid,
                                        beta_fid,
                                        snr_th,
                                        fmin,
                                        Tobs,
                                        det_Sc,
                                        based)

print('Time taken = {} seconds'.format(time.time() - starttime))

In [ ]:
np.save('data_mock_catalogues/m1z_'+detector+'_Ndet_%s_Nsamples_%s_twobox' % (n_detections,n_samples),m1z_mock_samples)
np.save('data_mock_catalogues/m2z_'+detector+'_Ndet_%s_Nsamples_%s_twobox' % (n_detections,n_samples),m2z_mock_samples)
np.save('data_mock_catalogues/dL_'+detector+'_Ndet_%s_Nsamples_%s_twobox' % (n_detections,n_samples),dL_mock_samples)
np.save('data_mock_catalogues/pdraw_'+detector+'_Ndet_%s_Nsamples_%s_twobox' % (n_detections,n_samples),pdraw_mock_samples)